<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/Streaming_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playing with structured input

Structured Stream PRocessing on CSV data...

In [1]:
#install software
# No clustering for this install !!
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 47.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=429638ae9fdee2d63ce7dd641e5cee277d3e3818ad9495354d8e3e0dab53dbcb
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
# get data
#from https://che-kulhan.medium.com/how-to-use-pyspark-streaming-with-google-colaboratory-d08ded30cabf
!wget https://raw.githubusercontent.com/lawlesst/vivo-sample-data/master/data/csv/people.csv

--2022-10-20 04:16:40--  https://raw.githubusercontent.com/lawlesst/vivo-sample-data/master/data/csv/people.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4095 (4.0K) [text/plain]
Saving to: ‘people.csv’

people.csv          100%[===================>]   4.00K  --.-KB/s    in 0s      

2022-10-20 04:16:40 (52.3 MB/s) - ‘people.csv’ saved [4095/4095]



In [3]:
# start the spark session

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("Structured input") \
    .getOrCreate()

In [4]:
!split -l 10 people.csv people-

In [5]:
from pyspark.sql.types import *
schema = StructType([
    StructField("person_ID",IntegerType(),True), 
    StructField("last", StringType(), True),
    StructField("middle", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("fax", StringType(), True),
    StructField("title", StringType(), True)])

In [6]:

!mkdir people
people_df = spark.readStream.format("csv")\
   .schema(schema) \
   .load("people")

print(people_df.isStreaming)

True


In [ ]:
results_df = people_df.select("*")
query2 = results_df.writeStream \
  .format("json") \
  .queryName("selectTable") \
  .option("checkpointLocation", "checkpoint") \
  .option("path", "results") \
  .outputMode("append") \
  .start() 

query2.awaitTermination()


Ok, previous task should run now and never end until you stop it...
Open the file view in the notebook and move people files to the people directory.

This triggers the streaming process. Results appears in the result directory.

When you understood. Interrupt the stream processing and clean running queries...

In [8]:
for q in spark.streams.active:
  print("stopping",q.name)
  q.stop()


stopping selectTable
